In [ ]:
import json
import time as nowtime
import torch

import torch
import torch.nn.functional as F
from torch_geometric.data import Data
from torch_geometric.nn import GraphConv, TopKPooling, GraphUNet, global_mean_pool, GATConv
import torch
from torch_geometric.data import InMemoryDataset, Data
from torch_geometric.utils import barabasi_albert_graph
import networkx as nx
import pickle as pkl
import random
import numpy as np
import torch_geometric.transforms as T
from typing import Optional, Callable

import numpy as np
from torch_geometric.nn import GCN
import torch.nn as nn
import torch
import torch.nn.functional as F
from sklearn.model_selection import train_test_split

from torch_geometric.nn import GCNConv,global_max_pool, GATConv, GATv2Conv,GraphSAGE,SAGEConv


from torch_geometric.nn import GraphConv,global_mean_pool,global_add_pool
from torch_geometric.loader import DataLoader

import numpy as np
import networkx as nx
import os
import math
from torch_geometric.transforms import NormalizeFeatures


import random

import numpy as np
import torch
from torch_geometric.nn import GCN

import torch
import torch.nn.functional as F
from sklearn.model_selection import train_test_split

from torch_geometric.nn import GCNConv,global_max_pool, GATConv, GATv2Conv,GraphSAGE,SAGEConv


from torch_geometric.nn import GraphConv,global_mean_pool,global_add_pool
from torch_geometric.loader import DataLoader
import torch.nn as nn
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence
import networkx as nx
import numpy as np 
import torch
from torch_geometric.data import Data

from torch.utils.data import DataLoader
import torch_geometric
import random
from torch.utils.data import DataLoader, Subset
from datetime import datetime
from scipy.stats import pearsonr
import matplotlib.pyplot as plt

import torch
import torch.nn.functional as F
from torch_geometric.data import Data
from torch_geometric.nn import GraphConv, TopKPooling, GraphUNet, global_mean_pool, GATConv

In [ ]:
def collate_fn(batch):
    names, namelen, tokns_list, graphs_list, labels,ori_key_list = zip(*batch)


    names_batch = torch.tensor(names, dtype=torch.long)
    namelen_batch = torch.tensor(namelen, dtype=torch.long)
    tokns_batch = torch_geometric.data.Batch.from_data_list(tokns_list)
    graphs_batch = torch_geometric.data.Batch.from_data_list(graphs_list)
    labels_batch = torch.stack([torch.tensor(l, dtype=torch.double) for l in labels], dim=0)
    method_batch= ori_key_list


    return names_batch, namelen_batch, tokns_batch, graphs_batch, labels_batch,method_batch





class BOWEncoder(nn.Module):
    def __init__(self, vocab_size, emb_size, hidden_size):
        super(BOWEncoder, self).__init__()
        self.emb_size=emb_size
        self.hidden_size = hidden_size
        self.embedding = nn.Embedding(vocab_size, emb_size)
        self.init_weights()
        
    def init_weights(self):
        nn.init.uniform_(self.embedding.weight, -0.1, 0.1)
        nn.init.constant_(self.embedding.weight[0], 0)
        
    def forward(self, input, input_len=None): 
        batch_size, seq_len =input.size()
        embedded = self.embedding(input)
        embedded= F.dropout(embedded, 0.25, self.training)
        
        # max pooling word vectors
        output_pool = F.max_pool1d(embedded.transpose(1,2), seq_len).squeeze(2) # [batch_size x emb_size]
        encoding = output_pool #torch.tanh(output_pool)        
        return encoding






class MyModel(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(MyModel, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)  
        self.relu = nn.ReLU()  
        self.fc2 = nn.Linear(hidden_size, output_size)  

    def forward(self, x):
        out = self.fc1(x.float())
        out = self.relu(out)
        out = self.fc2(out)
        return out


class MultiModel(torch.nn.Module):
    def __init__(self, in_channels, out_channels, hidden_channels, num_layers):
        super(MultiModel, self).__init__()
        pool_ratios = 0.6
        self.tree_conv1 = GraphUNet(13,out_channels ,hidden_channels,num_layers,pool_ratios)
        self.graph_conv1 = GraphUNet(20, out_channels, hidden_channels, num_layers, pool_ratios)
        

        vocab_size = 200000
        emb_size = 512
        hidden_size = 256
        self.encoder=BOWEncoder(vocab_size, emb_size, hidden_size)
        
        self.my_encoder=MyModel(16,hidden_size,emb_size)


        self.lin1=torch.nn.Linear(hidden_channels,out_channels)
        self.lin2=torch.nn.Linear(out_channels,512)
        self.hidden=512

        self.tree_gat1 = GATConv(hidden_channels, hidden_channels, heads=1, concat=False)
        self.graph_gat1 = GATConv(hidden_channels, hidden_channels, heads=1, concat=False)


        self.lin3 = torch.nn.Linear(hidden_channels, 512)

        lstm_dims=256
        n_hidden=512
        emb_size=512


        self.w_name = nn.Linear(2*lstm_dims, n_hidden)
        self.w_tok = nn.Linear(emb_size, n_hidden)
        self.w_graphseq=nn.Linear(2*lstm_dims, n_hidden)  
        self.w_desc = nn.Linear(2*lstm_dims, n_hidden)
        
        self.w_atten = nn.Linear(n_hidden, 1)
        self.fuse = nn.Linear(n_hidden*3,n_hidden)
        self.fuse4 = nn.Linear(n_hidden*4,n_hidden)
        self.init_weights()

        self.fc1 = nn.Linear(512, 128)
        self.fc2 = nn.Linear(128, 64)
        self.fc3 = nn.Linear(64, 1)
        self.fc4 = nn.Linear(512, 1)

        self.drop1=torch.nn.Dropout()
        self.drop2=torch.nn.Dropout()
    def init_weights(self):# Initialize Linear Weight 
        for m in [self.w_name, self.w_tok, self.w_graphseq, self.w_desc, self.w_atten, self.fuse]:        
            m.weight.data.uniform_(-0.1, 0.1)#nn.init.xavier_normal_(m.weight)
            nn.init.constant_(m.bias, 0.)


    def code_encoding(self, name_repr, tok_repr, graphseq_repr):

        batch_size=name_repr.shape[0]
       

        name_feat_hidden = self.w_name(name_repr).reshape(-1, self.hidden)
        tok_feat_hidden = self.w_tok(tok_repr).reshape(-1, self.hidden)
        graphseq_feat_hidden = self.w_graphseq(graphseq_repr).reshape(-1, self.hidden)
        name_attn_tanh = torch.tanh(name_feat_hidden)
        name_attn_scalar = self.w_atten(F.dropout(name_attn_tanh, 0.25 ,self.training).reshape(-1, self.hidden))            
        
        tok_attn_tanh = torch.tanh(tok_feat_hidden)
        tok_attn_scalar = self.w_atten(F.dropout(tok_attn_tanh, 0.25 ,self.training).reshape(-1, self.hidden))
        
        graphseq_attn_tanh = torch.tanh(graphseq_feat_hidden)
        graphseq_attn_scalar = self.w_atten(F.dropout(graphseq_attn_tanh, 0.25 ,self.training).reshape(-1, self.hidden))
        
        



        attn_cat = torch.cat([name_attn_scalar, tok_attn_scalar, graphseq_attn_scalar], 1)
        
        atten_weight = torch.sigmoid(attn_cat)


        name_feat_atten = torch.bmm(atten_weight[:,0].reshape(batch_size, 1, 1),name_repr.reshape(batch_size, 1, self.hidden))     
        tok_feat_atten = torch.bmm(atten_weight[:,0].reshape(batch_size, 1, 1),tok_repr.reshape(batch_size, 1, self.hidden))
        graphseq_feat_atten = torch.bmm(atten_weight[:,0].reshape(batch_size, 1, 1),graphseq_repr.reshape(batch_size, 1, self.hidden))

        cat_atten_repr = torch.cat((name_feat_atten, tok_feat_atten, graphseq_feat_atten), 2)
        code_repr = torch.tanh(self.fuse(F.dropout(cat_atten_repr, 0.25, training=self.training))).reshape(-1,self.hidden)
        return code_repr 

    def four_modal_fusion(self, name_repr, tok_repr, graphseq_repr, audio_repr):
        batch_size = name_repr.shape[0]

        name_feat_hidden = self.w_name(name_repr).reshape(-1, self.hidden)
        tok_feat_hidden = self.w_tok(tok_repr).reshape(-1, self.hidden)
        graphseq_feat_hidden = self.w_graphseq(graphseq_repr).reshape(-1, self.hidden)
        audio_feat_hidden = self.w_audio(audio_repr).reshape(-1, self.hidden)

        name_attn_tanh = torch.tanh(name_feat_hidden)
        name_attn_scalar = self.w_atten(F.dropout(name_attn_tanh, 0.25, self.training).reshape(-1, self.hidden))

        tok_attn_tanh = torch.tanh(tok_feat_hidden)
        tok_attn_scalar = self.w_atten(F.dropout(tok_attn_tanh, 0.25, self.training).reshape(-1, self.hidden))

        graphseq_attn_tanh = torch.tanh(graphseq_feat_hidden)
        graphseq_attn_scalar = self.w_atten(F.dropout(graphseq_attn_tanh, 0.25, self.training).reshape(-1, self.hidden))

        audio_attn_tanh = torch.tanh(audio_feat_hidden)
        audio_attn_scalar = self.w_atten(F.dropout(audio_attn_tanh, 0.25, self.training).reshape(-1, self.hidden))

        attn_cat = torch.cat([name_attn_scalar, tok_attn_scalar, graphseq_attn_scalar, audio_attn_scalar], 1)
        atten_weight = F.softmax(attn_cat, dim=1)

        name_feat_atten = torch.bmm(atten_weight[:, 0].reshape(batch_size, 1, 1), name_repr.reshape(batch_size, 1, self.hidden))
        tok_feat_atten = torch.bmm(atten_weight[:, 0].reshape(batch_size, 1, 1), tok_repr.reshape(batch_size, 1, self.hidden))
        graphseq_feat_atten = torch.bmm(atten_weight[:, 0].reshape(batch_size, 1, 1), graphseq_repr.reshape(batch_size, 1, self.hidden))
        audio_feat_atten = torch.bmm(atten_weight[:, 0].reshape(batch_size, 1, 1), audio_repr.reshape(batch_size, 1, self.hidden))

        cat_atten_repr = torch.cat((name_feat_atten, tok_feat_atten, graphseq_feat_atten, audio_feat_atten), 2)

        # Perform further fusion and get the final representation
        code_repr = torch.tanh(self.fuse(F.dropout(cat_atten_repr, 0.25, training=self.training))).reshape(-1, self.hidden)
        return code_repr

    def forward(self, name, namelen,tree_x,tree_edge_index,tree_batch,graph_x,graph_edge_index,graph_batch):
        

        name_repr=self.encoder(name,namelen) 

          
        # name_repr= self.my_encoder(name)




        tree_x=self.tree_conv1(tree_x,tree_edge_index,tree_batch)
        tree_x=self.tree_gat1(tree_x,tree_edge_index)
        tree_x=F.relu(tree_x)
        tree_x=self.drop1(tree_x)

        tree_x=self.lin3(tree_x)
        
        tree_x=global_mean_pool(tree_x,tree_batch)

        tok_repr = tree_x






        graph_x=self.graph_conv1(graph_x,graph_edge_index,graph_batch)
        graph_x=self.graph_gat1(graph_x,graph_edge_index)
        graph_x=F.relu(graph_x)
        graph_x=self.drop2(graph_x)

        graph_x=self.lin3(graph_x)
        graph_x=global_mean_pool(graph_x,graph_batch)
        
        graphseq_repr = graph_x
       

        # x=self.lin3(x)
        x=self.code_encoding(name_repr,tok_repr,graphseq_repr)

        x = nn.functional.relu(self.fc1(x))
        x = nn.functional.relu(self.fc2(x))

        
        return torch.tanh(self.fc3(x))  


    def my_sigm(x):
        
        return 1 / (1+torch.exp(-2*x))
    



    

In [ ]:
device = torch.device('cuda:3' if torch.cuda.is_available() else 'cpu')
        # device =  'cpu'
out_channels=256
hidden_channels=256
num_layers=3
model = MultiModel(1, out_channels, hidden_channels, num_layers).to(device)

In [ ]:
myname='crate'

model.load_state_dict(torch.load('/home/zhs/predict/0110model/qianyi/qianyimodel/0113no'+myname+'.pth'))


model.eval()

In [ ]:

json_path="/home/zhs/predict/expertcode/data/tn.json"
with open(json_path, "r") as file:
    data = json.load(file)


with open("/home/zhs/predict/model/project_keywords.json", "r") as file:
    project_keywords = json.load(file)


ori_key_list=[]
ori_body_list=[]
ori_len_list=[]
ori_tree_list=[]
ori_cfg_list=[]
ori_lables_list=[]


tree_batch = torch.load('/home/zhs/predict/expertcode/data/all_tree.pt')





cfg_batch = torch.load('/home/zhs/predict/expertcode/data/all_cfg.pt')

len(cfg_batch)
len(tree_batch)


time=1

for key in data.keys():

    value = json.loads(data[key])
    key=key.split("#")[0]

    
    

    if not  any(keyword in key for keyword in project_keywords[myname]):
        continue

    # print(key)
    
    size=value.get("len")
    body=value.get("indexList")
    lables=value.get("lables")
    # 要求lables 不是null
    if lables==None :
        continue

    d=0
    
    for tree in tree_batch:

        if tree.y==key:
            d=1
            # ori_tree_list.append(tree)
            t=tree
            continue
    # if d==0:
    #     print(key)
    c=0    
    for cfg in cfg_batch:
        if cfg.y==key:
            # print(key)
            c=1
            # ori_cfg_list.append(cfg)
            g=cfg
            continue
    if c==1 & d ==1:

        ori_key_list.append(key)
        ori_len_list.append(size)
        ori_body_list.append(body)
        ori_lables_list.append(lables)
        ori_cfg_list.append(g)
        ori_tree_list.append(t)
        
    time=time-1

names = ori_body_list

namelen =ori_len_list

seed = 42
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed_all(seed)






labels=list(map(float, ori_lables_list))



labels = np.array(labels)

normalized_labels = (labels - labels.min()) / (labels.max() - labels.min())



# print(normalized_labels)




log_array = [math.log(x + 1) for x in labels]


max_value = max(log_array)


normalized_array = [x / max_value for x in log_array]



labels=torch.tensor(normalized_labels)
# labels=list(map(float, ori_lables_list))
# print(labels)
labels=normalized_array


from torch.utils.data import Dataset, DataLoader

class MultiModalDataset(object):
    def __init__(self, names,namelen, tokns_list, graphs_list, labels,ori_key_list):

        self.names = names
        self.namelen=namelen
        self.tokns_list = tokns_list
        
        self.graphs_list = graphs_list
        
        self.labels = labels
        self.method = ori_key_list
    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        return self.names[idx] , self.namelen[idx] , self.tokns_list[idx] , self.graphs_list[idx] , self.labels[idx],self.method[idx]





# Assuming names, toks, graphseqs, labels are your data
dataset = MultiModalDataset(names,namelen, ori_tree_list, ori_cfg_list, labels,ori_key_list)



In [ ]:
print(len(names))
print(len(ori_cfg_list))
ori_key_list

In [ ]:

def train(train_loader):
     
    model.train()
    optimizer.zero_grad(device)

    total_loss = 0
    for names_batch, namelen_batch, tokns_batch, graphs_batch, labels_batch,method in train_loader:
        # print("starttrain")
        # print(method)
        names_batch=names_batch.to(device)
        namelen_batch=namelen_batch.to(device)
        tokns_batch=tokns_batch.to(device)
        graphs_batch=graphs_batch.to(device)
        labels_batch=labels_batch.to(device)
        # names_batch=names_batch.cuda()
        # namelen_batch=namelen_batch.cuda()
        # tokns_batch=tokns_batch.cuda()
        tokns_batch.x=tokns_batch.x.to(device)
        tokns_batch.edge_index=tokns_batch.edge_index.to(device)
        tokns_batch.batch=tokns_batch.batch.to(device)
        # graphs_batch=graphs_batch.cuda()
        # labels_batch=labels_batch.cuda()

        optimizer.zero_grad()
        output=model(names_batch,namelen_batch,
            tokns_batch.x,tokns_batch.edge_index,tokns_batch.batch,
            graphs_batch.x,graphs_batch.edge_index,graphs_batch.batch)


        y = labels_batch.view(-1, 1).float()
        loss = F.mse_loss(output, y)
        
        
        loss.backward()
        optimizer.step()
        
        total_loss += float(loss) * len(namelen_batch)
        

    return total_loss / len(train_loader.dataset)


@torch.no_grad()
def test(loader,my_size):
    model.eval()

    total_loss = 0
    preds = []
    reals = []
    for names_batch, namelen_batch, tokns_batch, graphs_batch, labels_batch ,method in loader:
        # print("starttest")
        start_time = nowtime.time()  
        names_batch=names_batch.to(device)
        namelen_batch=namelen_batch.to(device)
        
        tokns_batch.x=tokns_batch.x.to(device)
        tokns_batch.edge_index=tokns_batch.edge_index.to(device)
        tokns_batch.batch=tokns_batch.batch.to(device)

        graphs_batch.x=graphs_batch.x.to(device)
        graphs_batch.edge_index=graphs_batch.edge_index.to(device)
        graphs_batch.batch=graphs_batch.batch.to(device)

        labels_batch=labels_batch.to(device)
        
        output=model(names_batch,namelen_batch,
            tokns_batch.x,tokns_batch.edge_index,tokns_batch.batch,
            graphs_batch.x,graphs_batch.edge_index,graphs_batch.batch)
        

        end_time = nowtime.time()  
        eval_time = end_time - start_time  
        print(method)
        print(f"Evaluation time for one data point: {eval_time} seconds")

        # loss = F.mse_loss(out,torch.reshape(data.y, (data.y.shape[0],1)))
        y = labels_batch.view(-1, 1).float()



        
        loss = F.mse_loss(output, y)

        total_loss += float(loss) * len(namelen_batch)
        

        pred=output.detach().cpu().numpy()
        

        real=y.detach().cpu().numpy()
        
        preds = np.concatenate((preds,pred.flatten() ), axis=0)
        reals = np.concatenate((reals, real.flatten()), axis=0)

         
    # print("endtest")   
    return total_loss / len(loader.dataset) ,pearsonr(preds,reals)[0],preds,reals

In [ ]:

indices = list(range(len(dataset)))
random.shuffle(indices) 

test_indices = indices

print(test_indices)

test_dataset = Subset(dataset, test_indices)

batch_size = 1
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, collate_fn=collate_fn)
test_loss,prt,predt,realt = test(test_loader,0)
print(myname)
print("corr test  ",prt)

print(f'Epoch:  Test Loss: {test_loss:.4f}')